<a href="https://colab.research.google.com/github/DhanashriSaner/Cotton_Plant_Disease_Detection/blob/main/Cotton_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Dependencies**

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

**Set all the Constants**

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 617
CHANNELS=3
EPOCHS=50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import data into tensorflow dataset object**

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data/train",
shuffle=True,
image_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE

)

Found 1951 files belonging to 4 classes.


In [ ]:
class_names = train_data.class_names
class_names

['diseased cotton leaf',
 'diseased cotton plant',
 'fresh cotton leaf',
 'fresh cotton plant']

In [ ]:
validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data/val",
shuffle=True,
image_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE

)

Found 324 files belonging to 4 classes.


In [ ]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data/test",
shuffle=True,
image_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE

)

Found 36 files belonging to 5 classes.


**Cache, Shuffle, and Prefetch the Dataset**

In [ ]:
train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_data = validation_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

**Creating a Layer for Resizing and Normalization**
<hr>
Before we feed our images to network, we should be resizing it to the desired size. Moreover, to improve model performance, we should normalize the image pixel value (keeping them in range 0 and 1 by dividing by 256). This should happen while training as well as inference. Hence we can add that as a layer in our Sequential Model.

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

**Data Augmentation**

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

**Model Architecture**

In [38]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [39]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 617, 617, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 615, 615, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 307, 307, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 305, 305, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 152, 152, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 150, 150, 64)       

**Compiling the Model**
<hr>
We use adam Optimizer, SparseCategoricalCrossentropy for losses, accuracy as a metric

In [40]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)